In [5]:
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd

/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [123]:
house_df = pd.read_csv("../data/house_combined.csv")
suburb_list = house_df[['surburb','postcode']].drop_duplicates().reset_index().drop('index', axis = 1)

In [124]:
df2 = house_df[['surburb']].drop_duplicates().reset_index().drop('index', axis = 1)

In [125]:
suburb_postcode_df = house_df[['surburb','postcode']].drop_duplicates().reset_index().drop('index', axis = 1)

In [ ]:
list_1 = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        buildings = ox.geometries_from_place(place_name, tags = {'amenity': True, 'landuse':['retail','commercial']})
    except Exception:
        list_1[suburb] = None
        continue
    if len(buildings) == 0:
        list_1[suburb] = None
        continue
    for amenity, sub_df in buildings.groupby('amenity'):
        l[amenity] = len(sub_df)
    list_1[suburb] = l    

In [157]:
attribute_list = {**list_1, **list_2}
attribute_df = pd.DataFrame(attribute_list).T
attribute_df.to_csv('../data/raw/attribute_df.csv')
attribute_df = attribute_df[['bench','cafe','childcare','fast_food','kindergarten','parking','restaurant','place_of_worship','school','university']]

In [158]:
attribute_df

,bench,cafe,childcare,fast_food,kindergarten,parking,restaurant,place_of_worship,school,university
south melbourne,11.0,41.0,4.0,17.0,3.0,30.0,40.0,6.0,1.0,NaN
port melbourne,45.0,32.0,1.0,20.0,1.0,118.0,16.0,3.0,3.0,NaN
albert park,139.0,144.0,6.0,59.0,7.0,289.0,172.0,24.0,12.0,2.0
middle park,7.0,5.0,NaN,2.0,2.0,NaN,4.0,1.0,1.0,NaN
east melbourne,102.0,19.0,NaN,1.0,1.0,55.0,8.0,10.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
tallangatta,NaN,3.0,NaN,1.0,NaN,4.0,NaN,1.0,3.0,NaN
eskdale,NaN,1.0,NaN,NaN,NaN,1.0,NaN,2.0,1.0,NaN
dartmouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
beechworth,1.0,6.0,NaN,NaN,1.0,31.0,4.0,3.0,4.0,NaN


In [ ]:
leisure_list = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list[suburb] = l    

In [ ]:
leisure_list = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list[suburb] = l    

In [ ]:
leisure_list_2 = {}
for suburb in suburb_list['surburb'][500:]:
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list_2[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list_2[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list_2[suburb] = l    

In [376]:
leisure_attribute_list = {**leisure_list, **leisure_list_2}
leisure_attribute_list = pd.DataFrame(leisure_attribute_list).T
leisure_attribute_list.to_csv('../data/raw/leisure_df.csv')
leisure_attribute_list = leisure_attribute_list[['garden','park','playground','pitch']]

In [399]:
attribute_conbined = pd.concat([attribute_df,leisure_attribute_list ], axis = 1)

In [400]:
attribute_conbined = attribute_conbined.reset_index().join(suburb_postcode_df).drop('index',axis = 1)
attribute_conbined.to_csv('../data/attribute_conbined.csv')

Download Australia CPI data


In [62]:
xls = pd.ExcelFile('../data/cpi.xlsx')
df1 = pd.read_excel(xls, 'Data')[13:].drop('Unnamed: 0', axis = 1).rename({'Unnamed: 1':'year','Unnamed: 2':'cpi'},axis = 1).reset_index().drop('index',axis = 1)
cpi_list = list(df1['cpi'])
l = []
m = 0
for i in range(2019,2026):
    l.append([i,cpi_list[m]])
    m = m+1
cpi_df = pd.DataFrame(l, columns=['year','cpi'])
cpi_df.to_csv('../data/cpi.csv')

In [15]:
import pandas as pd
import openrouteservice
import math
house_df_combined = pd.read_csv('../data/house_combined.csv')
import ast
lat_list = []
long_list = []
for i in range(len(house_df_combined)):
    if pd.isnull(house_df_combined.geoLocation[i]):
        lat_list.append(None)
        long_list.append(None)
    else:
        latitude = ast.literal_eval(house_df_combined.geoLocation[i])['latitude']
        longitude = ast.literal_eval(house_df_combined.geoLocation[i])['longitude']
        lat_list.append(latitude)
        long_list.append(longitude)
house_df_combined['latitude'] = lat_list
house_df_combined['longitude'] = long_list

In [16]:
house_df_combined

,Unnamed: 0,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,...,postcode,surburb,year,month,price,bond,rent,state,latitude,longitude
0,0,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,...,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0,vic,-37.834163,144.960873
1,1,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2019-01-03T22:51:13.063Z,2018-10-02T00:00:38Z,...,3207,port melbourne,2019,1,$1800.00 per week,NaN,1800.0,vic,-37.844324,144.943722
2,2,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,2.0,1.0,2019-01-10T17:10:35.493Z,2018-10-05T04:57:13Z,...,3207,port melbourne,2019,1,$550.00 per week,NaN,550.0,vic,-37.843056,144.940260
3,3,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,0.0,2019-01-22T00:20:53.707Z,2018-10-05T07:07:14Z,...,3206,albert park,2019,1,$975.00 per week,NaN,975.0,vic,-37.846166,144.957026
4,4,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2019-01-17T17:10:07.873Z,2018-10-17T01:02:26Z,...,3207,port melbourne,2019,1,$500.00 per week,NaN,500.0,vic,-37.840834,144.939074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114980,116252,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2021-03-25T22:25:48.957Z,2021-03-12T06:10:56Z,...,3690,wodonga,2021,3,$215 per week,NaN,215.0,vic,-36.139450,146.882437
114981,116253,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,2.0,2021-03-31T00:18:22.6Z,2021-03-18T00:21:42Z,...,3691,baranduda,2021,3,$465 per week,NaN,465.0,vic,-36.178714,146.940579
114982,116254,['townhouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2021-03-30T22:40:44.093Z,2021-03-23T06:13:04Z,...,3690,wodonga,2021,3,$320 per week,NaN,320.0,vic,-36.129879,146.885637
114983,116255,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,2.0,2021-04-08T22:10:17.477Z,2021-03-26T00:07:22Z,...,3690,wodonga,2021,4,$360 per week,NaN,360.0,vic,-36.139852,146.887031


In [14]:
distance_list_6 = []
for i in range(92000,94000):
    print(i)
    if math.isnan(house_df_combined['longitude'][i]) is True:
        distance_list_6.append(None)
    else:
        coords = ((144.961602,-37.813937),(house_df_combined['longitude'][i],house_df_combined['latitude'][i]))
        client = openrouteservice.Client(key='5b3ce3597851110001cf6248faff1a0b834f4f2c83973e95065ad4bc') 
        routes = client.directions(coords, profile='driving-car', optimize_waypoints=True, radiuses = 1000)
        distance = routes['routes'][0]['summary']['distance']
        print(distance)
        distance_list_6.append(distance)

92000
40965.2
92001
57157.9
92002
27050.6
92003
28533.8
92004
22486.2
92005
23003.2
92006
24734.9
92007
33137.9
92008
34504.0
92009
26347.0
92010
40996.9
92011


KeyboardInterrupt: 